In [1]:
import pandas as pd
import numpy as np # linear algebra

# Replace 'path_to_dataset' with the actual path to your dataset file
df = pd.read_excel('Dataset2neww.xlsx')
df.shape

(4424, 29)

In [2]:
X = df.drop(columns=['Course'])
y = df['Course']

categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int', 'float']).columns

# Create DataFrames for categorical and numerical columns
X_categorical = X[categorical_columns]
X_numerical = X[numerical_columns]

from sklearn.preprocessing import LabelEncoder



# Perform label encoding only on columns with dtype object
label_encoder = LabelEncoder()
X_encoded = X_categorical.copy()
for col in categorical_columns:
    X_encoded[col] = X[col].astype(str)  # Ensure the column is of string dtype before label encoding
    X_encoded[col] = label_encoder.fit_transform(X_encoded[col])

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


# Initialize LabelEncoder for the target variable y
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)
# Merge with original X containing int and float columns
X_final = pd.concat([X_encoded, X.select_dtypes(include=['int', 'float'])], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_encoded, test_size=0.2, random_state=42)

In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

smote = SMOTE(random_state=42)
X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled_borderline, y_train_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)


## SMOTE + Bordeline SMOTE

RF

In [4]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the classifier
rf_classifier.fit(X_train_resampled_smote, y_train_resampled_smote)
# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with SMOTE): 0.488135593220339
Classification Report (Best Random Forest with SMOTE):
               precision    recall  f1-score   support

           0      0.250     0.200     0.222         5
           1      0.596     1.000     0.747        34
           2      0.771     0.844     0.806        32
           3      0.433     0.780     0.557        50
           4      0.410     0.410     0.410        39
           5      0.466     0.397     0.429        68
           6      0.265     0.545     0.356        33
           7      0.421     0.205     0.276        39
           8      0.382     0.328     0.353        64
           9      0.449     0.467     0.458        75
          10      0.234     0.208     0.220        53
          11      0.675     0.699     0.687       163
          12      0.188     0.130     0.154        23
       

In [5]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize Borderline SMOTE

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with Borderline SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with Borderline SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with Borderline SMOTE): 0.5005649717514125
Classification Report (Best Random Forest with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.567     1.000     0.723        34
           2      0.788     0.812     0.800        32
           3      0.433     0.780     0.557        50
           4      0.405     0.385     0.395        39
           5      0.491     0.382     0.430        68
           6      0.288     0.576     0.384        33
           7      0.526     0.256     0.345        39
           8      0.383     0.359     0.371        64
           9      0.462     0.480     0.471        75
          10      0.238     0.189     0.211        53
          11      0.659     0.712     0.684       163
          12      0.222     0.087     

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Gaussian NB

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 3.162e-05}
Accuracy (Best GaussianNB with SMOTE): 0.24858757062146894
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.020     0.200     0.036         5
           1      0.338     0.765     0.468        34
           2      0.472     0.531     0.500        32
           3      0.299     0.640     0.408        50
           4      0.133     0.051     0.074        39
           5      0.143     0.029     0.049        68
           6      0.273     0.182     0.218        33
           7      0.241     0.179     0.206        39
           8      0.222     0.031     0.055        64
           9      0.400     0.080     0.133        75
          10      0.179     0.189     0.183        53
          11      0.526     0.123     0.199       163
          12      0.105     0.087     0.095        23
          13      0.267  

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 3.162e-05}
Accuracy (Best GaussianNB with SMOTE): 0.26440677966101694
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.087     0.400     0.143         5
           1      0.316     0.882     0.465        34
           2      0.680     0.531     0.596        32
           3      0.333     0.660     0.443        50
           4      0.235     0.103     0.143        39
           5      0.056     0.015     0.023        68
           6      0.208     0.152     0.175        33
           7      0.226     0.179     0.200        39
           8      0.143     0.016     0.028        64
           9      0.412     0.093     0.152        75
          10      0.132     0.132     0.132        53
          11      0.533     0.098     0.166       163
          12      0.115     0.130     0.122        23
          13      0.235  

DT

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.37175141242937854
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.718     0.824     0.767        34
           2      0.765     0.812     0.788        32
           3      0.391     0.500     0.439        50
           4      0.209     0.231     0.220        39
           5      0.271     0.279     0.275        68
           6      0.269     0.424     0.329        33
           7      0.077     0.077     0.077        39
           8      0.227     0.234     0.231        64
           9      0.295     0.240     0.265        75
          10      0.089     0.075     0.082        53
          11      0.633     0.540     0.583       163
          12      0.211     0.174     0.190        23
          13      0.091     

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.3581920903954802
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.568     0.735     0.641        34
           2      0.767     0.719     0.742        32
           3      0.426     0.460     0.442        50
           4      0.091     0.103     0.096        39
           5      0.258     0.250     0.254        68
           6      0.277     0.394     0.325        33
           7      0.154     0.103     0.123        39
           8      0.197     0.219     0.207        64
           9      0.256     0.280     0.268        75
          10      0.196     0.208     0.202        53
          11      0.624     0.509     0.561       163
          12      0.292     0.304     0.298        23
          13      0.125     0

KNN


In [10]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with SMOTE): 0.34576271186440677
Classification Report (Best KNN with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.538     0.824     0.651        34
           2      0.576     0.594     0.585        32
           3      0.409     0.360     0.383        50
           4      0.182     0.205     0.193        39
           5      0.368     0.309     0.336        68
           6      0.347     0.515     0.415        33
           7      0.143     0.154     0.148        39
           8      0.241     0.219     0.230        64
           9      0.244     0.253     0.248        75
          10      0.183     0.208     0.195        53
          11      0.510     0.472     0.490       163
          12      0.174     0.174     0.174        23
          13      0.179     0.175     0.177      

In [11]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Borderline SMOTE to the training data only
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with Borderline SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with Borderline SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with Borderline SMOTE): 0.3446327683615819
Classification Report (Best KNN with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.562     0.794     0.659        34
           2      0.543     0.594     0.567        32
           3      0.444     0.400     0.421        50
           4      0.154     0.205     0.176        39
           5      0.387     0.353     0.369        68
           6      0.217     0.303     0.253        33
           7      0.154     0.154     0.154        39
           8      0.194     0.188     0.190        64
           9      0.273     0.280     0.276        75
          10      0.218     0.226     0.222        53
          11      0.516     0.491     0.503       163
          12      0.308     0.174     0.222        23
          13      0.156     

SVM

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE



# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with SMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with SMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with SMOTE): {'C': 100, 'gamma': 0.1}
Accuracy (SVM with SMOTE): 0.3954802259887006
Classification Report (SVM with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.636     0.824     0.718        34
           2      0.529     0.562     0.545        32
           3      0.532     0.500     0.515        50
           4      0.234     0.282     0.256        39
           5      0.296     0.471     0.364        68
           6      0.241     0.212     0.226        33
           7      0.192     0.128     0.154        39
           8      0.245     0.391     0.301        64
           9      0.346     0.360     0.353        75
          10      0.211     0.151     0.176        53
          11      0.608     0.638     0.623       163
          12      0.500     0.130     0.207        23
          13      0.083     0.075   

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with BorderlineSMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with BorderlineSMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with BorderlineSMOTE): {'C': 100, 'gamma': 0.1}
Accuracy (SVM with BorderlineSMOTE): 0.4
Classification Report (SVM with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.614     0.794     0.692        34
           2      0.545     0.562     0.554        32
           3      0.451     0.460     0.455        50
           4      0.239     0.282     0.259        39
           5      0.280     0.441     0.343        68
           6      0.323     0.303     0.312        33
           7      0.292     0.179     0.222        39
           8      0.236     0.406     0.299        64
           9      0.325     0.333     0.329        75
          10      0.243     0.170     0.200        53
          11      0.641     0.613     0.627       163
          12      0.444     0.174     0.250        23
          13      0.1

LR

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with SMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with SMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with SMOTE): {'C': 10, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with SMOTE): 0.3231638418079096
Classification Report (Logistic Regression with SMOTE):
               precision    recall  f1-score   support

           0      0.083     0.200     0.118         5
           1      0.433     0.765     0.553        34
           2      0.594     0.594     0.594        32
           3      0.388     0.520     0.444        50
           4      0.208     0.256     0.230        39
           5      0.292     0.206     0.241        68
           6      0.235     0.576     0.333        33
           7      0.191     0.231     0.209        39
           8      0.286     0.062     0.103        64
           9      0.243     0.333     0.281        75
          10      0.217     0.094     0.132        53
          11      0.591     0.337     0.430       163
          12      0.141     0.391     0

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with BorderlineSMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with BorderlineSMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with BorderlineSMOTE): {'C': 10, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with BorderlineSMOTE): 0.33220338983050846
Classification Report (Logistic Regression with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.500     0.200     0.286         5
           1      0.446     0.853     0.586        34
           2      0.649     0.750     0.696        32
           3      0.368     0.500     0.424        50
           4      0.171     0.179     0.175        39
           5      0.317     0.294     0.305        68
           6      0.227     0.515     0.315        33
           7      0.200     0.256     0.225        39
           8      0.154     0.031     0.052        64
           9      0.253     0.293     0.272        75
          10      0.219     0.132     0.165        53
          11      0.600     0.331     0.427       163
        

XGBoost

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': None}
Accuracy (XGBoost with SMOTE): 0.5028248587570622
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.711     0.941     0.810        34
           2      0.758     0.781     0.769        32
           3      0.538     0.560     0.549        50
           4      0.359     0.359     0.359        39
           5      0.444     0.471     0.457        68
           6      0.302     0.394     0.342        33
           7      0.318     0.179     0.230        39
           8      0.342     0.422     0.378        64
           9      0.434     0.480     0.456        75
          10      0.234     0.208     0.220        53
          11      0.709     0.779     0.743       163
          12      0.455     0.217     0.294        23
          13 

In [17]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': None}
Accuracy (XGBoost with SMOTE): 0.5299435028248588
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.660     0.912     0.765        34
           2      0.743     0.812     0.776        32
           3      0.547     0.700     0.614        50
           4      0.438     0.359     0.394        39
           5      0.465     0.485     0.475        68
           6      0.500     0.606     0.548        33
           7      0.333     0.205     0.254        39
           8      0.384     0.438     0.409        64
           9      0.407     0.467     0.435        75
          10      0.302     0.245     0.271        53
          11      0.744     0.785     0.764       163
          12      0.333     0.174     0.229        23
          13      0.273     0.300     0.286        40
          14      0.5

In [18]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost classifier with default parameters
xgb_classifier_baseline = XGBClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
xgb_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb_baseline = xgb_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_xgb_baseline = accuracy_score(y_test, y_pred_xgb_baseline)
classification_report_result_xgb_baseline = classification_report(y_test, y_pred_xgb_baseline, digits=3)

# Print results
print("Baseline XGBoost Model:")
print(f"Accuracy: {accuracy_xgb_baseline}")
print("Classification Report:\n", classification_report_result_xgb_baseline)


Baseline XGBoost Model:
Accuracy: 0.5389830508474577
Classification Report:
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.721     0.912     0.805        34
           2      0.788     0.812     0.800        32
           3      0.635     0.660     0.647        50
           4      0.410     0.410     0.410        39
           5      0.537     0.529     0.533        68
           6      0.378     0.424     0.400        33
           7      0.455     0.256     0.328        39
           8      0.354     0.453     0.397        64
           9      0.391     0.453     0.420        75
          10      0.310     0.245     0.274        53
          11      0.697     0.859     0.769       163
          12      0.600     0.130     0.214        23
          13      0.341     0.350     0.346        40
          14      0.482     0.346     0.403        78
          15      0.323     0.244     0.278        41
    

AdaBoost

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 200}
Accuracy (AdaBoost with SMOTE): 0.23728813559322035
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.000     0.000     0.000        34
           2      0.531     0.812     0.642        32
           3      0.284     0.380     0.325        50
           4      0.095     0.103     0.099        39
           5      0.250     0.074     0.114        68
           6      0.128     0.455     0.200        33
           7      0.179     0.128     0.149        39
           8      0.000     0.000     0.000        64
           9      0.178     0.400     0.246        75
          10      0.000     0.000     0.000        53
          11      0.410     0.294     0.343       163
          12      0.105     0.348     0.162        23
 

In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 150}
Accuracy (AdaBoost with SMOTE): 0.22372881355932203
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.000     0.000     0.000        34
           2      0.491     0.812     0.612        32
           3      0.162     0.220     0.186        50
           4      0.133     0.051     0.074        39
           5      0.222     0.059     0.093        68
           6      0.152     0.758     0.253        33
           7      0.115     0.077     0.092        39
           8      0.045     0.016     0.023        64
           9      0.165     0.293     0.212        75
          10      0.000     0.000     0.000        53
          11      0.347     0.264     0.300       163
          12      0.115     0.304     0.167        23
 

In [21]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the AdaBoost classifier with default parameters
ada_classifier_baseline = AdaBoostClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
ada_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ada_baseline = ada_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_ada_baseline = accuracy_score(y_test, y_pred_ada_baseline)
classification_report_result_ada_baseline = classification_report(y_test, y_pred_ada_baseline, digits=3)

# Print results
print("Baseline AdaBoost Model:")
print(f"Accuracy: {accuracy_ada_baseline}")
print("Classification Report:\n", classification_report_result_ada_baseline)


Baseline AdaBoost Model:
Accuracy: 0.29265536723163843
Classification Report:
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.420     0.853     0.563        34
           2      0.727     0.250     0.372        32
           3      0.000     0.000     0.000        50
           4      0.240     0.154     0.188        39
           5      0.111     0.044     0.063        68
           6      0.000     0.000     0.000        33
           7      0.000     0.000     0.000        39
           8      0.000     0.000     0.000        64
           9      0.274     0.227     0.248        75
          10      0.000     0.000     0.000        53
          11      0.246     0.926     0.389       163
          12      0.000     0.000     0.000        23
          13      0.000     0.000     0.000        40
          14      0.000     0.000     0.000        78
          15      0.000     0.000     0.000        41
  

CatBoost

In [22]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Assuming X_train_resampled_smote, y_train_resampled_smote, X_test, y_test are defined

# Initialize the CatBoost classifier with default parameters
catboost_classifier = CatBoostClassifier(random_state=42)


# Fit the CatBoost classifier
catboost_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_catboost = catboost_classifier.predict(X_test)

# Evaluate the model
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
classification_report_catboost = classification_report(y_test, y_pred_catboost, digits=3)

# Print results
print(f"Accuracy (CatBoost Baseline with SMOTE): {accuracy_catboost}")
print("Classification Report (CatBoost Baseline with SMOTE):\n", classification_report_catboost)


Learning rate set to 0.084395
0:	learn: 2.6977135	total: 241ms	remaining: 4m
1:	learn: 2.6050272	total: 304ms	remaining: 2m 31s
2:	learn: 2.5173510	total: 320ms	remaining: 1m 46s
3:	learn: 2.4428661	total: 348ms	remaining: 1m 26s
4:	learn: 2.3832337	total: 371ms	remaining: 1m 13s
5:	learn: 2.3329843	total: 387ms	remaining: 1m 4s
6:	learn: 2.2803336	total: 403ms	remaining: 57.2s
7:	learn: 2.2415799	total: 419ms	remaining: 52s
8:	learn: 2.2003765	total: 436ms	remaining: 48s
9:	learn: 2.1671782	total: 453ms	remaining: 44.8s
10:	learn: 2.1312836	total: 481ms	remaining: 43.3s
11:	learn: 2.1023409	total: 498ms	remaining: 41s
12:	learn: 2.0679342	total: 514ms	remaining: 39s
13:	learn: 2.0437755	total: 530ms	remaining: 37.3s
14:	learn: 2.0196276	total: 546ms	remaining: 35.9s
15:	learn: 1.9930072	total: 562ms	remaining: 34.6s
16:	learn: 1.9719326	total: 579ms	remaining: 33.5s
17:	learn: 1.9493597	total: 595ms	remaining: 32.5s
18:	learn: 1.9296229	total: 612ms	remaining: 31.6s
19:	learn: 1.91176

475:	learn: 0.5743129	total: 8.33s	remaining: 9.17s
476:	learn: 0.5734044	total: 8.35s	remaining: 9.15s
477:	learn: 0.5725844	total: 8.36s	remaining: 9.13s
478:	learn: 0.5713555	total: 8.38s	remaining: 9.12s
479:	learn: 0.5697796	total: 8.4s	remaining: 9.1s
480:	learn: 0.5688437	total: 8.41s	remaining: 9.08s
481:	learn: 0.5679706	total: 8.43s	remaining: 9.06s
482:	learn: 0.5673815	total: 8.45s	remaining: 9.04s
483:	learn: 0.5662961	total: 8.46s	remaining: 9.02s
484:	learn: 0.5649933	total: 8.48s	remaining: 9s
485:	learn: 0.5644725	total: 8.5s	remaining: 8.98s
486:	learn: 0.5635424	total: 8.51s	remaining: 8.97s
487:	learn: 0.5625567	total: 8.53s	remaining: 8.95s
488:	learn: 0.5610474	total: 8.55s	remaining: 8.94s
489:	learn: 0.5592903	total: 8.57s	remaining: 8.92s
490:	learn: 0.5579590	total: 8.59s	remaining: 8.9s
491:	learn: 0.5568600	total: 8.6s	remaining: 8.88s
492:	learn: 0.5557688	total: 8.62s	remaining: 8.87s
493:	learn: 0.5551522	total: 8.64s	remaining: 8.85s
494:	learn: 0.554526

LGBM

In [23]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002877 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2004
[LightGBM] [Info] Number of data points in the train set: 10251, number of used features: 28
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start tra

In [24]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003772 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1982
[LightGBM] [Info] Number of data points in the train set: 9655, number of used features: 28
[LightGBM] [Info] Start training from score -7.229321
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] St